In [1]:
import os
import pandas as pd
import numpy as np
import torch
import datetime as dt
import torch.nn as nn
from torchdiffeq import odeint_adjoint as odeint
import csv
from pathlib import Path
from scipy import signal
from ipywidgets import interact
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import random

import torch.nn.functional as F

In [67]:
df = pd.read_csv(os.path.join(os.getcwd(), "results", "Full_experts_check.csv")).set_index("ID")
icps = []
abps = []
for row in df.iterrows():
    row = row[1]
    icp = row["icp_full"][1:-1].replace(",","").split(" ")
    icp = [float(value.strip("\n")) for value in icp if value is not ""]
    icps.append(icp)
#     print(len(icp))
    abp = row["abp_full"][1:-1].replace(",","").split(" ")
    abp = [float(value.strip("\n")) for value in abp if value is not ""]
#     print(len(abp))
    abps.append(abp)
ICP_full = torch.tensor(icps ,dtype=torch.float)
ABP_full = torch.tensor(abps,dtype=torch.float)

In [102]:
excluded_prefix = "pretraining"
prefixes = [ "_Training_Artificial_AE_", "_Training_", "__", "_pretrained_"]
suffixes = ["unfrozen", "pretrained"]
folders = os.listdir(os.path.join(os.getcwd(),"experiments"))

In [96]:
def find_model_info(name, experiments):
    for experiment in experiments:
        if name == experiment["name_full"]:
            return {
                "model": experiment["model"],
                "o2pfcn": experiment["manager"]["o2p_fcn"],
                "input_preprocessing": experiment["manager"]["input_preprocessing"],
                "siamese": experiment["dataset"]["siamese"]
            }

In [105]:
import re
output_dataframe_dict = {}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for folder in tqdm(folders):
    experiment_name = folder
    name = re.sub(r'\d+-\d+-\d+', '', folder)
    name = re.sub(r'\d+', '', name)
    if excluded_prefix not in name:
        for prefix in prefixes:
            name = name.replace(prefix,"")
        for suffix in suffixes:
            name = name.replace(suffix,"")
        name = name.strip("_")
        model_info = find_model_info(name, experiments)
        model_path = os.path.join(os.getcwd(), "experiments", experiment_name, "model_best.pth")
        model = model_info["model"].to(device)
        model.load_state_dict(torch.load(model_path, map_location=device)["state_dict"])
        model.eval()
        
        if model_info["siamese"]:
            model_input = [ICP_full.to(device), ABP_full.to(device)]
        else:
            model_input = [ICP_full.to(device)]
            
        if model_info["input_preprocessing"] is not None:
            input_preprocessing = model_info["input_preprocessing"]
        else:
            input_preprocessing = lambda x: x
        
        if model_info["o2pfcn"] is not None:
            output_to_pred_fcn = model_info["o2pfcn"]
        else:
            output_to_pred_fcn = lambda x: x
        
        model_input = input_preprocessing(model_input)
        outputs = model(*model_input)
        predicted = output_to_pred_fcn(outputs)
        output_dataframe_dict[experiment_name] = predicted

  4%|███▌                                                                               | 2/46 [00:00<00:02, 17.85it/s]

SiameseResNet
SiameseResNet


  9%|███████▏                                                                           | 4/46 [00:00<00:03, 12.39it/s]

SiameseResNet
FullyConnected
FullyConnected_DualChannel
LSTMFCN
ResNet_DualChannel
SiameseResNet

 20%|████████████████▏                                                                  | 9/46 [00:00<00:02, 14.94it/s]

...

 87%|███████████████████████████████████████████████████████████████████████▎          | 40/46 [00:05<00:01,  4.10it/s]

ResNet
ResNet_DualChannel
ResNet_DualChannel_Multilabel
ResNet_Multilabel
SiameseNeuralODE
SiameseNeuralODE_Multilabel


 96%|██████████████████████████████████████████████████████████████████████████████▍   | 44/46 [00:09<00:01,  1.99it/s]

SiameseResNet
SiameseResNet_Multilabel


100%|██████████████████████████████████████████████████████████████████████████████████| 46/46 [00:09<00:00,  4.66it/s]


In [117]:
output_dataframe_dict["ID"] = list(range(len(df)))
for key in output_dataframe_dict.keys():
    data = output_dataframe_dict[key]
    if not isinstance(data, list):
#         output_dataframe_dict[key] = numpy.split(data, 650)
        if len(data.shape) > 1:
            output_dataframe_dict[key] = [a[0] for a in np.split(data, 650)]
df_pred = pd.DataFrame(output_dataframe_dict).set_index("ID")

In [123]:
scoring_df = df.join(df_pred)
scoring_df = scoring_df.rename(columns = {"Likely type": "Likely", "Possible type": "Possible"}, inplace = False)

In [124]:
scoring_df

,Likely,Possible,abp_full,icp_full,2020-10-08_pretrained_SiameseResNet_pretrained_1,2020-10-08_pretrained_SiameseResNet_pretrained_unfrozen_1,2020-10-08__SiameseResNet_1,2020-10-11_Training_FullyConnected_1,2020-10-11_Training_FullyConnected_DualChannel_1,2020-10-11_Training_LSTMFCN_1,...,2020-10-13_Training_Artificial_AE_ODE_DualChannel_Multilabel_1,2020-10-13_Training_Artificial_AE_ODE_Multilabel_1,2020-10-13_Training_Artificial_AE_ResNet_1,2020-10-13_Training_Artificial_AE_ResNet_DualChannel_1,2020-10-13_Training_Artificial_AE_ResNet_DualChannel_Multilabel_1,2020-10-13_Training_Artificial_AE_ResNet_Multilabel_1,2020-10-13_Training_Artificial_AE_SiameseNeuralODE_1,2020-10-13_Training_Artificial_AE_SiameseNeuralODE_Multilabel_1,2020-10-13_Training_Artificial_AE_SiameseResNet_1,2020-10-13_Training_Artificial_AE_SiameseResNet_Multilabel_1
ID,,,,,,,,,,,,,,,,,,,,,
0,1,1,[3.04814486e-02 2.18967845e-02 1.42104387e-02 ...,[5.81779680e-03 1.87093060e-03 0.00000000e+00 ...,0,0,0,0,0,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0]",0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0]",0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,"[1.0, 0.0, 0.0, 0.0, 0.0]"
1,2,2,[0.02428143 0.01584578 0.00807451 0.00233634 0...,[6.47123462e-02 5.54119309e-02 4.46266755e-02 ...,1,2,2,1,1,1,...,"[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]",2,2,"[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]",2,"[0.0, 0.0, 1.0, 0.0, 0.0]",2,"[0.0, 0.0, 1.0, 0.0, 0.0]"
2,2,2,[1.59646774e-01 1.43772435e-01 1.28992707e-01 ...,[0.01877641 0.01091759 0.00506625 0.00137589 0...,1,4,4,3,1,1,...,"[0.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 1.0, 0.0, 0.0, 0.0]",1,4,"[0.0, 0.0, 0.0, 0.0, 1.0]","[0.0, 1.0, 0.0, 0.0, 0.0]",4,"[0.0, 0.0, 0.0, 1.0, 1.0]",1,"[0.0, 1.0, 0.0, 0.0, 1.0]"
3,1,1,[5.52120121e-02 4.58446053e-02 3.61205810e-02 ...,[2.67341992e-02 2.06881025e-02 1.48638903e-02 ...,0,0,0,0,0,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0]","[1.0, 0.0, 0.0, 0.0, 0.0]",4,4,"[1.0, 0.0, 0.0, 0.0, 1.0]","[1.0, 0.0, 0.0, 0.0, 0.0]",0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,"[1.0, 0.0, 0.0, 0.0, 0.0]"
4,3,3,[0.04620486 0.06037026 0.07319366 0.08527039 0...,[1.48506940e-02 1.06446098e-02 6.36478001e-03 ...,1,2,2,2,1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]",2,2,"[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 1.0]",2,"[0.0, 0.0, 1.0, 0.0, 0.0]",2,"[0.0, 0.0, 1.0, 1.0, 0.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,2,2,"[0.0017795, 0.0, 0.01063416, 0.01229615, 0.018...","[0.0, 0.00363864, 0.01151878, 0.02390274, 0.04...",1,1,1,1,1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]",1,1,"[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]",1,"[0.0, 1.0, 0.0, 0.0, 0.0]",1,"[0.0, 1.0, 0.0, 0.0, 0.0]"
646,3,3,"[0.0, 0.00449792, 0.00149089, 0.02160732, 0.03...","[0.03117772, 0.03105301, 0.03702201, 0.0494947...",1,1,1,2,2,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0]","[0.0, 1.0, 0.0, 0.0, 0.0]",2,2,"[0.0, 0.0, 1.0, 0.0, 0.0]","[0.0, 0.0, 1.0, 0.0, 0.0]",1,"[0.0, 0.0, 0.0, 0.0, 0.0]",1,"[0.0, 0.0, 0.0, 0.0, 1.0]"
647,2,4,"[0.00843174, 0.00791627, 0.00437937, 0.0046104...","[0.0, 0.0012543, 0.00611003, 0.01476069, 0.027...",1,3,0,1,1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 0.0, 0.0, 0.0]",3,3,"[0.0, 0.0, 0.0, 1.0, 0.0]","[0.0, 0.0, 0.0, 1.0, 0.0]",0,"[0.0, 0.0, 0.0, 1.0, 0.0]",0,"[1.0, 0.0, 0.0, 0.0, 0.0]"


In [141]:
rights = []
names = scoring_df.columns[5:]
for i in range(len(names)):
    right = 0
    if "Multilabel" in names[i]:
        true_labels = scoring_df["Likely"].values
        possible_labels = scoring_df["Possible"].values
        predicted_labels = scoring_df[names[i]].values
        for true, possible, predicted in zip(true_labels, possible_labels, predicted_labels):
            if predicted[true-1] == 1 or predicted[possible-1] == 1:
                right+=1
    else:
        right = len(scoring_df.loc[scoring_df["Likely"] == scoring_df[names[i]] + 1])
        right += len(scoring_df.loc[scoring_df["Possible"] == scoring_df[names[i]] + 1].loc[scoring_df["Possible"] != scoring_df["Likely"]])
#         right = len(scoring_df.query('`Likely` == `' + names[i] + '` + 1 or `Possible` == `' + names[i] + '` + 1'))     
    rights.append(right)

In [142]:
for name,right in zip(names, rights):
    print("{} : {:2.2f} %".format(name, right/650*100))

2020-10-08_pretrained_SiameseResNet_pretrained_unfrozen_1 : 82.62 %
2020-10-08__SiameseResNet_1 : 83.54 %
2020-10-11_Training_FullyConnected_1 : 72.46 %
2020-10-11_Training_FullyConnected_DualChannel_1 : 70.92 %
2020-10-11_Training_LSTMFCN_1 : 80.62 %
2020-10-11_Training_ResNet_DualChannel_1 : 81.69 %
2020-10-11_Training_SiameseResNet_1 : 80.92 %
2020-10-12_Training_CNN_1 : 81.54 %
2020-10-12_Training_CNN_DualChannel_1 : 76.46 %
2020-10-12_Training_CNN_DualChannel_Multilabel_1 : 75.69 %
2020-10-12_Training_CNN_Multilabel_1 : 78.62 %
2020-10-12_Training_FullyConnected_DualChannel_Multilabel_1 : 58.46 %
2020-10-12_Training_FullyConnected_Multilabel_1 : 56.62 %
2020-10-12_Training_LSTMFCN_DualChannel_1 : 79.54 %
2020-10-12_Training_LSTMFCN_DualChannel_Multilabel_1 : 72.31 %
2020-10-12_Training_LSTMFCN_Multilabel_1 : 75.08 %
2020-10-12_Training_ResNet_1 : 86.00 %
2020-10-12_Training_ResNet_DualChannel_Multilabel_1 : 82.92 %
2020-10-12_Training_ResNet_Multilabel_1 : 85.69 %
2020-10-12_Train

In [146]:
save_df = {"names": names,
          "best_accuracy": np.array(rights)/650*100}
pd.DataFrame(save_df).to_csv(os.path.join(os.getcwd(), "results", "final_experts_scoring_best.csv"))

In [138]:
print(max(rights)/650)

0.86


In [147]:
rights = []
names = scoring_df.columns[5:]
for i in range(len(names)):
    right = 0
    if "Multilabel" in names[i]:
        true_labels = scoring_df["Likely"].values
        possible_labels = scoring_df["Possible"].values
        predicted_labels = scoring_df[names[i]].values
        for true, possible, predicted in zip(true_labels, possible_labels, predicted_labels):
            if predicted[true-1] == 1 and predicted[possible-1] == 1:
                right+=1
    else:
        right = len(scoring_df.loc[scoring_df["Likely"] == scoring_df[names[i]] + 1])
#         right += len(scoring_df.loc[scoring_df["Possible"] == scoring_df[names[i]] + 1].loc[scoring_df["Possible"] != scoring_df["Likely"]])
#         right = len(scoring_df.query('`Likely` == `' + names[i] + '` + 1 or `Possible` == `' + names[i] + '` + 1'))     
    rights.append(right)

In [148]:
for name,right in zip(names, rights):
    print("{} : {:2.2f} %".format(name, right/650*100))

2020-10-08_pretrained_SiameseResNet_pretrained_unfrozen_1 : 80.62 %
2020-10-08__SiameseResNet_1 : 80.62 %
2020-10-11_Training_FullyConnected_1 : 69.38 %
2020-10-11_Training_FullyConnected_DualChannel_1 : 69.38 %
2020-10-11_Training_LSTMFCN_1 : 77.69 %
2020-10-11_Training_ResNet_DualChannel_1 : 78.92 %
2020-10-11_Training_SiameseResNet_1 : 79.54 %
2020-10-12_Training_CNN_1 : 79.38 %
2020-10-12_Training_CNN_DualChannel_1 : 74.62 %
2020-10-12_Training_CNN_DualChannel_Multilabel_1 : 68.46 %
2020-10-12_Training_CNN_Multilabel_1 : 71.85 %
2020-10-12_Training_FullyConnected_DualChannel_Multilabel_1 : 55.38 %
2020-10-12_Training_FullyConnected_Multilabel_1 : 54.15 %
2020-10-12_Training_LSTMFCN_DualChannel_1 : 76.31 %
2020-10-12_Training_LSTMFCN_DualChannel_Multilabel_1 : 68.15 %
2020-10-12_Training_LSTMFCN_Multilabel_1 : 70.46 %
2020-10-12_Training_ResNet_1 : 81.85 %
2020-10-12_Training_ResNet_DualChannel_Multilabel_1 : 76.77 %
2020-10-12_Training_ResNet_Multilabel_1 : 80.00 %
2020-10-12_Train

In [149]:
save_df = {"names": names,
          "strict_accuracy": np.array(rights)/650*100}
pd.DataFrame(save_df).to_csv(os.path.join(os.getcwd(), "results", "final_experts_scoring_strict.csv"))

In [93]:
import torch
from torch import nn
import pickle
import datetime as dt
from experiment_manager import Manager
from models.CNNmodel import CNN
from models.FCmodel import FCmodel  
from models.RNNmodel import LSTMFCN
from models.SiameseModels import SiameseNeuralODE, SiameseResNet
from models.ResnetODEmodels import ODE, ResNet
from utils import resampling_dataset_loader, Memory_efficient_loader, learning_rate_with_decay
import pandas as pd
import numpy as np
from torchvision.transforms import Compose, Lambda
from sklearn.metrics import accuracy_score, f1_score, jaccard_score
from metrics import best_accuracy_ml

length = 100
test_dataset = None
train_dateset = None

experiments = [
    {
        "model": SiameseResNet(5, ae=False),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "SiameseResNet",
        "criterion": nn.CrossEntropyLoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "Accuracy": accuracy_score,
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="weighted")
            },
            "o2p_fcn": lambda x: np.argmax(x.cpu().detach().numpy(), axis=1),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": None,
            "leading_metric": "Accuracy",
            "input_preprocessing": None
        },
        "dataset": {
            "siamese": True,
            "multilabel": False,
            "include_artificial_ae": True,
            "transforms": None
        }
    },

    {
        "model": FCmodel(180, 5, ae=False),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "FullyConnected",
        "criterion": nn.CrossEntropyLoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "Accuracy": accuracy_score,
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="weighted")
            },
            "o2p_fcn": lambda x: np.argmax(x.cpu().detach().numpy(), axis=1),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": None,
            "leading_metric": "Accuracy",
            "input_preprocessing": None
        },
        "dataset": {
            "siamese": False,
            "multilabel": False,
            "include_artificial_ae": True,
            "transforms": None
        }
    },

    {
        "model": LSTMFCN(180, 5, ae=False),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "LSTMFCN",
        "criterion": nn.CrossEntropyLoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "Accuracy": accuracy_score,
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="weighted")
            },
            "o2p_fcn": lambda x: np.argmax(x.cpu().detach().numpy(), axis=1),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": None,
            "leading_metric": "Accuracy",
            "input_preprocessing": None
        },
        "dataset": {
            "siamese": False,
            "multilabel": False,
            "include_artificial_ae": True,
            "transforms": None
        }
    },
    {
        "model": ResNet(5),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "ResNet",
        "criterion": nn.CrossEntropyLoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "Accuracy": accuracy_score,
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="weighted")
            },
            "o2p_fcn": lambda x: np.argmax(x.cpu().detach().numpy(), axis=1),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": None,
            "leading_metric": "Accuracy",
            "input_preprocessing": lambda x: [a.unsqueeze(1) for a in x]
        },
        "dataset": {
            "siamese": False,
            "multilabel": False,
            "include_artificial_ae": True,
            "transforms": None
        }
    },
    {
        "model": CNN(5),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "CNN",
        "criterion": nn.CrossEntropyLoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "Accuracy": accuracy_score,
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="weighted")
            },
            "o2p_fcn": lambda x: np.argmax(x.cpu().detach().numpy(), axis=1),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": None,
            "leading_metric": "Accuracy",
            "input_preprocessing": lambda x: [a.unsqueeze(1) for a in x]
        },
        "dataset": {
            "siamese": False,
            "multilabel": False,
            "include_artificial_ae": True,
            "transforms": None
        }
    },
    {
        "model": FCmodel(360, 5, ae=False),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "FullyConnected_DualChannel",
        "criterion": nn.CrossEntropyLoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "Accuracy": accuracy_score,
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="weighted")
            },
            "o2p_fcn": lambda x: np.argmax(x.cpu().detach().numpy(), axis=1),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": None,
            "leading_metric": "Accuracy",
            "input_preprocessing": lambda x: [torch.cat(x, dim=1)]
        },
        "dataset": {
            "siamese": True,
            "multilabel": False,
            "include_artificial_ae": True,
            "transforms": None
        }
    },

    {
        "model": LSTMFCN(180, 5, channels=[2, 32, 64, 32], ae=False),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "LSTMFCN_DualChannel",
        "criterion": nn.CrossEntropyLoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "Accuracy": accuracy_score,
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="weighted")
            },
            "o2p_fcn": lambda x: np.argmax(x.cpu().detach().numpy(), axis=1),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": None,
            "leading_metric": "Accuracy",
            "input_preprocessing": lambda x: [torch.stack(x, dim=1)]
        },
        "dataset": {
            "siamese": True,
            "multilabel": False,
            "include_artificial_ae": True,
            "transforms": None
        }
    },
    {
        "model": ResNet(5, in_channels=2),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "ResNet_DualChannel",
        "criterion": nn.CrossEntropyLoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "Accuracy": accuracy_score,
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="weighted")
            },
            "o2p_fcn": lambda x: np.argmax(x.cpu().detach().numpy(), axis=1),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": None,
            "leading_metric": "Accuracy",
            "input_preprocessing": lambda x: [torch.stack(x, dim=1)]
        },
        "dataset": {
            "siamese": True,
            "multilabel": False,
            "include_artificial_ae": True,
            "transforms": None
        }
    },
    {
        "model": CNN(5, channels=[2, 32, 64, 64]),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "CNN_DualChannel",
        "criterion": nn.CrossEntropyLoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "Accuracy": accuracy_score,
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="weighted")
            },
            "o2p_fcn": lambda x: np.argmax(x.cpu().detach().numpy(), axis=1),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": None,
            "leading_metric": "Accuracy",
            "input_preprocessing": lambda x: [torch.stack(x, dim=1)]
        },
        "dataset": {
            "siamese": True,
            "multilabel": False,
            "include_artificial_ae": True,
            "transforms": None
        }
    },
    {
        "model": SiameseResNet(5, ae=False),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "SiameseResNet_Multilabel",
        "criterion": nn.BCELoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="macro"),
                "Jaccard": lambda labels, preds: jaccard_score(labels, preds, average="macro"),
                "Best_accuracy": best_accuracy_ml
            },
            "o2p_fcn": lambda x: np.where(torch.sigmoid(x.cpu().detach()).numpy() >= 0.5, np.ones(x.shape), np.zeros(x.shape)),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": lambda x: torch.sigmoid(x),
            "leading_metric": "F1_Score",
            "input_preprocessing": None
        },
        "dataset": {
            "siamese": True,
            "multilabel": True,
            "include_artificial_ae": True,
            "transforms": None
        }
    },

    {
        "model": FCmodel(180, 5, ae=False),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "FullyConnected_Multilabel",
        "criterion": nn.BCELoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="macro"),
                "Jaccard": lambda labels, preds: jaccard_score(labels, preds, average="macro"),
                "Best_accuracy": best_accuracy_ml
            },
            "o2p_fcn": lambda x: np.where(torch.sigmoid(x.cpu().detach()).numpy() >= 0.5, np.ones(x.shape), np.zeros(x.shape)),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": lambda x: torch.sigmoid(x),
            "leading_metric": "F1_Score",
            "input_preprocessing": None
        },
        "dataset": {
            "siamese": False,
            "multilabel": True,
            "include_artificial_ae": True,
            "transforms": None
        }
    },

    {
        "model": LSTMFCN(180, 5, ae=False),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "LSTMFCN_Multilabel",
        "criterion": nn.BCELoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="macro"),
                "Jaccard": lambda labels, preds: jaccard_score(labels, preds, average="macro"),
                "Best_accuracy": best_accuracy_ml
            },
            "o2p_fcn": lambda x: np.where(torch.sigmoid(x.cpu().detach()).numpy() >= 0.5, np.ones(x.shape), np.zeros(x.shape)),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": lambda x: torch.sigmoid(x),
            "leading_metric": "F1_Score",
            "input_preprocessing": None
        },
        "dataset": {
            "siamese": False,
            "multilabel": True,
            "include_artificial_ae": True,
            "transforms": None
        }
    },
    {
        "model": ResNet(5),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "ResNet_Multilabel",
        "criterion": nn.BCELoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="macro"),
                "Jaccard": lambda labels, preds: jaccard_score(labels, preds, average="macro"),
                "Best_accuracy": best_accuracy_ml
            },
            "o2p_fcn": lambda x: np.where(torch.sigmoid(x.cpu().detach()).numpy() >= 0.5, np.ones(x.shape), np.zeros(x.shape)),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": lambda x: torch.sigmoid(x),
            "leading_metric": "F1_Score",
            "input_preprocessing": lambda x: [a.unsqueeze(1) for a in x]
        },
        "dataset": {
            "siamese": False,
            "multilabel": True,
            "include_artificial_ae": True,
            "transforms": None
        }
    },
    {
        "model": CNN(5),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "CNN_Multilabel",
        "criterion": nn.BCELoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="macro"),
                "Jaccard": lambda labels, preds: jaccard_score(labels, preds, average="macro"),
                "Best_accuracy": best_accuracy_ml
            },
            "o2p_fcn": lambda x: np.where(torch.sigmoid(x.cpu().detach()).numpy() >= 0.5, np.ones(x.shape), np.zeros(x.shape)),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": lambda x: torch.sigmoid(x),
            "leading_metric": "F1_Score",
            "input_preprocessing": lambda x: [a.unsqueeze(1) for a in x]
        },
        "dataset": {
            "siamese": False,
            "multilabel": True,
            "include_artificial_ae": True,
            "transforms": None
        }
    },
    {
        "model": FCmodel(360, 5, ae=False),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "FullyConnected_DualChannel_Multilabel",
        "criterion": nn.BCELoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="macro"),
                "Jaccard": lambda labels, preds: jaccard_score(labels, preds, average="macro"),
                "Best_accuracy": best_accuracy_ml
            },
            "o2p_fcn": lambda x: np.where(torch.sigmoid(x.cpu().detach()).numpy() >= 0.5, np.ones(x.shape), np.zeros(x.shape)),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": lambda x: torch.sigmoid(x),
            "leading_metric": "F1_Score",
            "input_preprocessing": lambda x: [torch.cat(x, dim=1)]
        },
        "dataset": {
            "siamese": True,
            "multilabel": True,
            "include_artificial_ae": True,
            "transforms": None
        }
    },

    {
        "model": LSTMFCN(180, 5, channels=[2, 32, 64, 32], ae=False),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "LSTMFCN_DualChannel_Multilabel",
        "criterion": nn.BCELoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="macro"),
                "Jaccard": lambda labels, preds: jaccard_score(labels, preds, average="macro"),
                "Best_accuracy": best_accuracy_ml
            },
            "o2p_fcn": lambda x: np.where(torch.sigmoid(x.cpu().detach()).numpy() >= 0.5, np.ones(x.shape), np.zeros(x.shape)),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": lambda x: torch.sigmoid(x),
            "leading_metric": "F1_Score",
            "input_preprocessing": lambda x: [torch.stack(x, dim=1)]
        },
        "dataset": {
            "siamese": True,
            "multilabel": True,
            "include_artificial_ae": True,
            "transforms": None
        }
    },
    {
        "model": ResNet(5, in_channels=2),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "ResNet_DualChannel_Multilabel",
        "criterion": nn.BCELoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="macro"),
                "Jaccard": lambda labels, preds: jaccard_score(labels, preds, average="macro"),
                "Best_accuracy": best_accuracy_ml
            },
            "o2p_fcn": lambda x: np.where(torch.sigmoid(x.cpu().detach()).numpy() >= 0.5, np.ones(x.shape), np.zeros(x.shape)),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": lambda x: torch.sigmoid(x),
            "leading_metric": "F1_Score",
            "input_preprocessing": lambda x: [torch.stack(x, dim=1)]
        },
        "dataset": {
            "siamese": True,
            "multilabel": True,
            "include_artificial_ae": True,
            "transforms": None
        }
    },
    {
        "model": CNN(5, channels=[2, 32, 64, 64]),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "CNN_DualChannel_Multilabel",
        "criterion": nn.BCELoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="macro"),
                "Jaccard": lambda labels, preds: jaccard_score(labels, preds, average="macro"),
                "Best_accuracy": best_accuracy_ml
            },
            "o2p_fcn": lambda x: np.where(torch.sigmoid(x.cpu().detach()).numpy() >= 0.5, np.ones(x.shape), np.zeros(x.shape)),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": False,
            "loss_preprocessing": lambda x: torch.sigmoid(x),
            "leading_metric": "F1_Score",
            "input_preprocessing": lambda x: [torch.stack(x, dim=1)]
        },
        "dataset": {
            "siamese": True,
            "multilabel": True,
            "include_artificial_ae": True,
            "transforms": None
        }
    },
    {
        "model": SiameseNeuralODE(5, ae=False),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "SiameseNeuralODE",
        "criterion": nn.CrossEntropyLoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "Accuracy": accuracy_score,
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="weighted")
            },
            "o2p_fcn": lambda x: np.argmax(x.cpu().detach().numpy(), axis=1),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": True,
            "loss_preprocessing": None,
            "leading_metric": "Accuracy",
            "input_preprocessing": None
        },
        "dataset": {
            "siamese": True,
            "multilabel": False,
            "include_artificial_ae": False,
            "transforms": None
        }
    },
    {
        "model": SiameseNeuralODE(5, ae=False),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "SiameseNeuralODE_Multilabel",
        "criterion": nn.BCELoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="macro"),
                "Jaccard": lambda labels, preds: jaccard_score(labels, preds, average="macro"),
                "Best_accuracy": best_accuracy_ml
            },
            "o2p_fcn": lambda x: np.where(torch.sigmoid(x.cpu().detach()).numpy() >= 0.5, np.ones(x.shape), np.zeros(x.shape)),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": True,
            "loss_preprocessing": lambda x: torch.sigmoid(x),
            "leading_metric": "F1_Score",
            "input_preprocessing": None
        },
        "dataset": {
            "siamese": True,
            "multilabel": True,
            "include_artificial_ae": False,
            "transforms": None
        }
    },
    {
        "model": ODE(5, in_channels=2),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "ODE_DualChannel_Multilabel",
        "criterion": nn.BCELoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="macro"),
                "Jaccard": lambda labels, preds: jaccard_score(labels, preds, average="macro"),
                "Best_accuracy": best_accuracy_ml
            },
            "o2p_fcn": lambda x: np.where(torch.sigmoid(x.cpu().detach()).numpy() >= 0.5, np.ones(x.shape), np.zeros(x.shape)),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": True,
            "loss_preprocessing": lambda x: torch.sigmoid(x),
            "leading_metric": "F1_Score",
            "input_preprocessing": lambda x: [torch.stack(x, dim=1)]
        },
        "dataset": {
            "siamese": True,
            "multilabel": True,
            "include_artificial_ae": False,
            "transforms": None
        }
    },
    {
        "model": ODE(5),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "ODE_Multilabel",
        "criterion": nn.BCELoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="macro"),
                "Jaccard": lambda labels, preds: jaccard_score(labels, preds, average="macro"),
                "Best_accuracy": best_accuracy_ml
            },
            "o2p_fcn": lambda x: np.where(torch.sigmoid(x.cpu().detach()).numpy() >= 0.5, np.ones(x.shape), np.zeros(x.shape)),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": True,
            "loss_preprocessing": lambda x: torch.sigmoid(x),
            "leading_metric": "F1_Score",
            "input_preprocessing": lambda x: [a.unsqueeze(1) for a in x]
        },
        "dataset": {
            "siamese": False,
            "multilabel": True,
            "include_artificial_ae": False,
            "transforms": None
        }
    },
    {
        "model": ODE(5),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "ODE",
        "criterion": nn.CrossEntropyLoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "Accuracy": accuracy_score,
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="weighted")
            },
            "o2p_fcn": lambda x: np.argmax(x.cpu().detach().numpy(), axis=1),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": True,
            "loss_preprocessing": None,
            "leading_metric": "Accuracy",
            "input_preprocessing": lambda x: [a.unsqueeze(1) for a in x]
        },
        "dataset": {
            "siamese": False,
            "multilabel": False,
            "include_artificial_ae": True,
            "transforms": None
        }
    },
    {
        "model": ODE(5, in_channels=2),
        "pretrained": False,
        "pretraining_path": None,
        "name_full": "ODE_DualChannel",
        "criterion": nn.CrossEntropyLoss(),
        "optimizer": lambda x: torch.optim.SGD(x, lr=0.01, momentum=0.95, nesterov=True),
        "scheduler": learning_rate_with_decay(0.01, 256, 256, length, boundary_epochs=[33, 66], decay_rates=[1, 0.1, 0.01]),
        "manager": {
            "pretraining": False,
            "metrics": {
                "Accuracy": accuracy_score,
                "F1_Score": lambda labels, preds: f1_score(labels, preds, average="weighted")
            },
            "o2p_fcn": lambda x: np.argmax(x.cpu().detach().numpy(), axis=1),
            "test_dataset": test_dataset,
            "loader_size": 256,
            "normalize": True,
            "nfe_logging": True,
            "loss_preprocessing": None,
            "leading_metric": "Accuracy",
            "input_preprocessing": lambda x: [torch.stack(x, dim=1)]
        },
        "dataset": {
            "siamese": True,
            "multilabel": False,
            "include_artificial_ae": True,
            "transforms": None
        }
    }
    ]